In [2]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [3]:
tr_mmc_codes = pd.read_csv("/content/drive/MyDrive/data/tr_mcc_codes.csv", sep=";")
tr_types = pd.read_csv("/content/drive/MyDrive/data/tr_types.csv", sep=";")
transactions = pd.read_csv("/content/drive/MyDrive/data/transactions.csv", sep=",", nrows=1000000)
gender_train = pd.read_csv("/content/drive/MyDrive/data/gender_train.csv", sep=",")

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [4]:
tr_type_data = transactions['tr_type'].sample(n=1000)
atm_or_pos = tr_types[tr_types['tr_description'].str.contains('POS') | tr_types['tr_description'].str.contains('АТМ')]
tr_type_data[tr_type_data.isin(atm_or_pos['tr_type'])].size / 1000

0.654

## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [5]:
frequency = transactions.groupby(['tr_type']).size() / transactions['tr_type'].size
top10 = frequency.sort_values(ascending=False).head(10)
top10_description = tr_types[tr_types['tr_type'].isin(top10.index)]
frequency, top10, top10_description

(tr_type
 1000    0.000008
 1010    0.231117
 1030    0.118975
 1100    0.015476
 1110    0.137658
           ...   
 7074    0.000505
 7075    0.000012
 8100    0.000002
 8145    0.000008
 8146    0.000003
 Length: 72, dtype: float64,
 tr_type
 1010    0.231117
 2010    0.151166
 7070    0.149006
 1110    0.137658
 1030    0.118975
 2370    0.049830
 7010    0.028972
 7030    0.026078
 7071    0.015781
 1100    0.015476
 dtype: float64,
      tr_type                                     tr_description
 39      7010       Взнос наличных через АТМ (в своем тер.банке)
 47      7030  Перевод на карту (с карты) через АТМ (в предел...
 59      7070  Перевод на карту (с карты) через Мобильный бан...
 60      7071  Перевод на карту (с карты) через Мобильный бан...
 98      1010                              Покупка. POS ТУ СБ РФ
 99      1030                     Оплата услуги. Банкоматы СБ РФ
 100     1100                                Покупка. ТУ  Россия
 101     1110                         

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [6]:
customers_amounts = transactions[['customer_id', 'amount']]
positive_amounts = customers_amounts[customers_amounts['amount'] >= 0]
summ_of_customers_amounts = positive_amounts.groupby(['customer_id']).sum()
highest_income = summ_of_customers_amounts.sort_values('amount', ascending=False).head(1)
highest_income

,amount
customer_id,
70780820,1.248115e+09


In [7]:
customers_amounts = transactions[['customer_id', 'amount']]
negative_amounts = customers_amounts[customers_amounts['amount'] < 0]
summ_of_customers_amounts = negative_amounts.groupby(['customer_id']).sum()
highest_expenses = summ_of_customers_amounts.sort_values('amount', ascending=True).head(1)
highest_expenses

,amount
customer_id,
70780820,-1.249952e+09


In [8]:
abs(highest_expenses['amount']) - abs(highest_income['amount'])

customer_id
70780820    1837317.98
Name: amount, dtype: float64

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [9]:
top10_transactions = transactions[transactions['tr_type'].isin(top10_description['tr_type'])]
mean = top10_transactions.groupby(['tr_type'])['amount'].mean()
median = top10_transactions.groupby(['tr_type'])['amount'].median()
mean, median

(tr_type
 1010    -19784.748640
 1030     -5320.980222
 1100    -44061.827262
 1110    -32119.330371
 2010   -136077.629325
 2370   -205418.249032
 7010    276391.789596
 7030     86104.332909
 7070     65569.831700
 7071     66806.826623
 Name: amount, dtype: float64,
 tr_type
 1010     -7411.52
 1030     -2245.92
 1100    -10188.26
 1110    -11207.57
 2010    -44918.32
 2370    -44918.32
 7010    112295.79
 7030     13951.52
 7070     11319.42
 7071      3593.47
 Name: amount, dtype: float64)

In [10]:
client_transactions = transactions[transactions['customer_id'].isin(highest_income.index.values)]
mean = client_transactions.groupby(['tr_type'])['amount'].mean()
median = client_transactions.groupby(['tr_type'])['amount'].median()
mean, median

(tr_type
 2010   -2.941056e+06
 2330   -2.382398e+06
 2370   -2.218505e+06
 6110    1.756293e+04
 7020    1.465129e+04
 7021    2.850067e+04
 7030    1.406196e+04
 7031    1.921036e+04
 7034    1.430648e+04
 7040    2.153833e+04
 7070    1.384821e+04
 7071    1.860057e+04
 7074    2.281159e+04
 Name: amount, dtype: float64,
 tr_type
 2010   -3368873.660
 2330   -2245915.770
 2370   -2245915.770
 6110      10028.010
 7020      10319.985
 7021      28500.670
 7030       8848.910
 7031      12992.620
 7034      14306.480
 7040      14396.320
 7070       8714.150
 7071      12543.440
 7074       9387.930
 Name: amount, dtype: float64)

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [11]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mmc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,"mcc_description,,,,",tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ
...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [12]:
customers_amounts = transactions[['gender', 'amount']]
positive_amounts = customers_amounts[customers_amounts['amount'] < 0]
mean_amounts_of_expenses_genders = positive_amounts.groupby(['gender'])['amount'].mean()
abs(mean_amounts_of_expenses_genders[0] - mean_amounts_of_expenses_genders[1])

32718.054920224655

In [13]:
customers_amounts = transactions[['gender', 'amount']]
positive_amounts = customers_amounts[customers_amounts['amount'] >= 0]
mean_amounts_of_income_genders = positive_amounts.groupby(['gender'])['amount'].mean()
abs(mean_amounts_of_income_genders[0] - mean_amounts_of_income_genders[1])

63366.57104801464

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [14]:
positive_amounts_transactions = transactions[transactions['amount'] >= 0]
incomes_of_genders = positive_amounts_transactions.groupby(['gender', 'tr_type'])['amount'].sum('amount')
smallest_income_gender0 = incomes_of_genders[0].sort_values().head(10)
smallest_income_gender1 = incomes_of_genders[1].sort_values().head(10)
smallest_income_gender0, smallest_income_gender1

(tr_type
 4051     1122.96
 4210     2245.92
 4110     2245.92
 2370     2335.75
 7075     6737.75
 2110    22459.16
 1010    31532.66
 4100    40334.88
 2210    68315.82
 1210    83644.87
 Name: amount, dtype: float64,
 tr_type
 2020      2245.92
 4110      4491.84
 1010     35317.03
 8100     64682.37
 6000     79280.83
 7041     87590.72
 2370    113194.15
 7015    121279.45
 2110    179673.26
 2010    224591.59
 Name: amount, dtype: float64)

In [15]:
smallest_income_gender0[smallest_income_gender0.index.isin(smallest_income_gender1.index)].index

Int64Index([4110, 2370, 2110, 1010], dtype='int64', name='tr_type')

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [16]:
negative_amounts = transactions[transactions['amount'] < 0]
mmc_codes_gender = negative_amounts.groupby(['gender', 'mcc_code'])['amount'].sum('amount')
mmc_codes_gender

gender  mcc_code
0.0     742          -386207.68
        1711         -674897.69
        1799         -261626.73
        2741          -20746.42
        3000       -74431645.27
                       ...     
1.0     8999       -28643346.63
        9222         -988205.17
        9311         -373271.22
        9399        -1810820.67
        9402           -4581.67
Name: amount, Length: 349, dtype: float64

In [17]:
gender0 = mmc_codes_gender[0]
gender1 = mmc_codes_gender[1]
modul = abs(gender0 - gender1).sort_values(ascending=False).head(10)
tr_mmc_codes[tr_mmc_codes['mcc_code'].isin(modul.index)]

,mcc_code,"mcc_description,,,,"
22,4829,"Денежные переводы,,,,"
62,5511,"Легковой и грузовой транспорт — продажа,сервис..."
64,5533,"Автозапчасти и аксессуары,,,,"
65,5541,"Станции техобслуживания,,,,"
75,5661,"Обувные магазины,,,,"
90,5812,"Места общественного питания,рестораны,,,"
116,5977,"Магазины косметики,,,,"
123,6010,Финансовые институты — снятие наличности вручн...
124,6011,Финансовые институты — снятие наличности автом...
162,7995,"Транзакции по азартным играм,,,,"


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [18]:
transactions[['tr_day', 'tr_time']] = transactions['tr_datetime'].str.split(' ', expand= True)
transactions[['tr_hours', 'tr_minutes', 'tr_seconds']] = transactions['tr_time'].str.split(':', expand= True).astype(int)
transactions['tr_hours']

0         10
1          7
2          7
3          8
4         14
          ..
999579    23
999580    19
999581    22
999582    23
999583    23
Name: tr_hours, Length: 999584, dtype: int64

In [19]:
negative_amounts = transactions[transactions['amount'] < 0]
count_of_transactions_in_night = negative_amounts[negative_amounts['tr_hours'] < 6].groupby('gender').size()
count_of_transactions_in_night

gender
0.0    38827
1.0    43017
dtype: int64

## Задание за защиту

Исключите из данных транзакции со следующими mcc кодами 6011, 4829, 6010, 5541

По каждому mcc коду посчитайте количество транзакций. Исключите из данных транзакции с пятью mcc самыми редкими кодами.

По оставшимся транзакциям посчитайте то же самое, что считали в задании 7. В датафрейм добавьте ещё один столбец с указанием пола, представители которого тратят по соответствующей категории больше.

In [20]:
mmc_codes = [6011, 4829, 6010, 5541]
new_transactions = transactions['mcc_code'].isin(mmc_codes)
new_transactions = new_transactions.apply(lambda x: not x)
new_transactions = transactions[new_transactions]
new_transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,"mcc_description,,,,",tr_description,tr_day,tr_time,tr_hours,tr_minutes,tr_seconds
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ,0,10:23:26,10,23,26
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ,6,07:08:31,7,8,31
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ,8,07:06:10,7,6,10
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ,11,08:49:03,8,49,3
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов,считывающих ...",Оплата услуги. Банкоматы СБ РФ,11,14:12:08,14,12,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ,18,23:38:60,23,38,60
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ,206,19:00:10,19,0,10
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ,90,22:40:57,22,40,57
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии,авиакомпании,,,",Возврат покупки. ТУ СБ РФ,104,23:39:35,23,39,35


In [21]:
count_of_transactions = new_transactions.groupby('mcc_code').size()
rare_transactions = count_of_transactions.sort_values().head(5)
new_count_of_transactions = count_of_transactions.isin(rare_transactions)
new_count_of_transactions = new_count_of_transactions.apply(lambda x: not x)
new_count_of_transactions = count_of_transactions[new_count_of_transactions]
new_count_of_transactions

mcc_code
742       27
1711      18
1731       4
1799      57
2741       8
        ... 
8699       6
8999    3126
9222      47
9311      38
9399     190
Length: 175, dtype: int64

In [22]:
new_transactions = transactions[transactions['mcc_code'].isin(new_count_of_transactions.index)]

negative_amounts = new_transactions[new_transactions['amount'] < 0]
mmc_codes_gender = negative_amounts.groupby(['gender', 'mcc_code'])['amount'].sum('amount')
mmc_codes_gender

gender  mcc_code
0.0     742          -386207.68
        1711         -674897.69
        1799         -261626.73
        2741          -20746.42
        3000       -74431645.27
                       ...     
1.0     8699         -444183.07
        8999       -28643346.63
        9222         -988205.17
        9311         -373271.22
        9399        -1810820.67
Name: amount, Length: 336, dtype: float64

In [23]:
gender0 = mmc_codes_gender[0]
gender1 = mmc_codes_gender[1]
modul = abs(gender0 - gender1).sort_values(ascending=False).head(10)
tr_mmc_codes[tr_mmc_codes['mcc_code'].isin(modul.index)]
gender0 = mmc_codes_gender[0].reindex(tr_mmc_codes['mcc_code'])
gender1 = mmc_codes_gender[1].reindex(tr_mmc_codes['mcc_code'])
gender0 = gender0.fillna(0)
gender1 = gender1.fillna(0)
tr_mmc_codes['main_gender'] = np.where(gender0 > gender1, "0", "1")
tr_mmc_codes

,mcc_code,"mcc_description,,,,",main_gender
0,742,"Ветеринарные услуги,,,,",1
1,1711,"Генеральные подрядчики по вентиляции,теплоснаб...",1
2,1731,"Подрядчики по электричеству,,,,",0
3,1799,"Подрядчики,специализированная торговля — нигде...",0
4,2741,"Разнообразные издательства/печатное дело,,,,",0
...,...,...,...
179,9211,"Судовые выплаты,включая алименты и детскую под...",1
180,9222,"Штрафы,,,,",0
181,9311,"Налоговые платежи,,,,",1
182,9399,"Правительственные услуги,нигде ранее не класси...",0
